<h1>ETL</h1>

In [1]:
import pandas as pd
import gzip

<h3>Dataset "steam_games"</h3>

In [2]:
steam_games = r"..\Datasets\steam_games\steam_games.json.gz"

# Cargamos el archivo comprimido en un DataFrame:
df_steam_games = pd.read_json(gzip.open(steam_games, 'rt', encoding='utf-8'), lines=True)
df_steam_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [3]:
# Eliminamos los registros en blanco:
df_sg = df_steam_games.dropna()

In [4]:
# Eliminamos las columnas que no serán  utilizadas:
df_sg = df_sg.drop(["publisher", "title", "url", "tags", "reviews_url", "specs", "price", "early_access"], axis=1)

In [5]:
df_sg.head()

,genres,app_name,release_date,id,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,761140.0,Kotoshiro
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,643980.0,Secret Level SRL
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,670290.0,Poolians.com
88313,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,767400.0,彼岸领域
88315,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,772540.0,Trickjump Games Ltd


Tratamiento de la columna "genres"

In [6]:
# Rellenamos los valores faltantes con una lista vacía:
df_sg["genres"] = df_sg["genres"].fillna("[]")

# Convertimos las listas de géneros en cadenas separadas por comas:
df_sg["genres"] = df_sg["genres"].apply(lambda x: ", ".join(x))

# Creamos dummies para los géneros:
genres_dummies = df_sg["genres"].str.get_dummies(", ")

# Concatenamos los dummies al DataFrame original
df_sg = pd.concat([df_sg, genres_dummies], axis=1)

# Eliminamos la columna "genres":
del df_sg["genres"]

In [7]:
df_sg.head()

,app_name,release_date,id,developer,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
88310,Lost Summoner Kitty,2018-01-04,761140.0,Kotoshiro,1,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0
88311,Ironbound,2018-01-04,643980.0,Secret Level SRL,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
88312,Real Pool 3D - Poolians,2017-07-24,670290.0,Poolians.com,0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,0
88313,弹炸人2222,2017-12-07,767400.0,彼岸领域,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
88315,Battle Royale Trainer,2018-01-04,772540.0,Trickjump Games Ltd,1,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


Tratamiento de la columna "release_date"

In [8]:
# Convertimos la columna "release_date" en tipo datetime:
df_sg["release_date"] = pd.to_datetime(df_sg["release_date"], errors='coerce')

# Eliminamos los valores vacíos:
df_sg = df_sg.dropna(subset=["release_date"])

# Extraemos el año y creamos una nueva columna 'release_year'
df_sg.insert(1, "release_year", df_sg["release_date"].dt.year)

# Eliminamos del DataFrame la columna "release_date"
del df_sg["release_date"]

Tratamiento de la columna "id"

In [9]:
# Revisamos la presencia de duplicados:
df_sg[df_sg["id"].duplicated()]

,app_name,release_year,id,developer,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
102883,Wolfenstein II: The New Colossus,2017,612880.0,Machine Games,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# Revisamos el caso duplicado:
df_sg[df_sg["id"] == 612880.0]

,app_name,release_year,id,developer,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
102204,Wolfenstein II: The New Colossus,2017,612880.0,Machine Games,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102883,Wolfenstein II: The New Colossus,2017,612880.0,Machine Games,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Al tratarse de un registro repetido, eliminamos el duplicado:
df_sg.drop_duplicates(subset=["id"], inplace=True)

In [12]:
# Convertimos la columna "id" en tipo int:
df_sg["id"] = df_sg["id"].astype(int)

In [13]:
# Renombramos a "item_id":
df_sg = df_sg.rename(columns={"id": "item_id"})

In [14]:
df_sg.head()

,app_name,release_year,item_id,developer,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
88310,Lost Summoner Kitty,2018,761140,Kotoshiro,1,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0
88311,Ironbound,2018,643980,Secret Level SRL,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
88312,Real Pool 3D - Poolians,2017,670290,Poolians.com,0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,0
88313,弹炸人2222,2017,767400,彼岸领域,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
88315,Battle Royale Trainer,2018,772540,Trickjump Games Ltd,1,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


Capitalizamos y quitamos espacios en las columnas "str"

In [19]:
df_sg["app_name"] = df_sg["app_name"].astype(str).str.capitalize().str.strip()
df_sg["developer"] = df_sg["developer"].astype(str).str.capitalize().str.strip()

In [24]:
df_sg[["app_name", "developer"]].head()

,app_name,developer
88310,Lost summoner kitty,Kotoshiro
88311,Ironbound,Secret level srl
88312,Real pool 3d - poolians,Poolians.com
88313,弹炸人2222,彼岸领域
88315,Battle royale trainer,Trickjump games ltd


Guardado como CSV

In [25]:
df_sg.to_csv(r"..\Archivos Producidos\games.csv", index=False)